# Machine Learning, Composability, and the Category Learn
A neural network, in practice, is often interpreted and constructed as a composition of multiple predefined components that serve as "atoms" from which to build increasingly complexl models. Naturally, machine learning libraries often feature these components as built-ins, but this implicitly uses the composability of these "trainable" components.
(Fong et al., 2019) formalise this notion by describing "trainable" functions as morphisms in a suitably constructed category, the category **Learn**. Here, we introduce their concept of a *learner* by constructing a small neural network within Julia.

In [1]:
using Random

abstract type Learner{M, N} end

## Learners
The problem of supervised learning is to find suitable approximation for a target function $f^* : A \to B$ within a collection of functions $f : P \times A \to B$ where $P$ is a parameter space. Precisely, by looking at examples $(a, b) \in A \times B$ with $f^*(a) \approx b$, we want to discover a choice of parameters $p^* \in P$ such that $f^* \approx f(p^*, \cdot)$ according to some error metric. (Fong et al., 2019) constructs *learners* to contain the necessary information to carry out this task, including:
 - Parameter Space: $P$
 - Implementation Function: $I : P \times A \to B$
 - Update Function: $U : P \times A \times B \to P$, and
 - Request Function: $r : P \times A \times B \to A$.

Intuitively, a learner can be throught of as a family of parameterised functions equipped with a method for updating its current parameters according to obsevations.

### What a Learner Does (Implementation Functions)
First, to specify a learner from an input space $A$ to an output space $B$, we need to describe the relationship it implements between these inputs and outputs. This relationship is allowed to depend on parameters taken from the learner's parameter space $P$ and is captured by its implementation function $I : P \times A \to B$. We view the implementation function as carrying a parameter $p \in P$ and an input $a \in A$ to a predicted output $I(p, a) \in B$.

Consider, for instance, the simple example of a fully-connected linear layer from $\mathbb{R}^M$ to $\mathbb{R}^N$ that has:
 - a parameter space $P = \mathbb{R}^{N \times M} \times \mathbb{R}^N$ containing all $N \times M$ weight matrices and all $N \times 1$ bias vectors, and
 - an implementation function where, for any parameters $p = (W, b) \in P$, we have
$$ I(p, x) = W x + b $$
   for all $x \in \mathbb{R}^M$.

We can implement this learner in Julia as a type `LinearLearner{M, N}` where instances of this type represent choices of parameters.

In [2]:
struct LinearLearner{M, N} <: Learner{M, N}
    weights::Matrix{Float64}
    biases::Vector{Float64}
    
    function LinearLearner{M, N}(weights, biases) where {M, N}
        @assert size(weights) == (N, M)
        @assert size(biases) == (N,)
        return new(weights, biases)
    end
end

function LinearLearner{M, N}() where {M, N}
    weights = rand(N, M)
    biases = rand(N)
    return LinearLearner{M, N}(weights, biases)
end

function implementation(learner::LinearLearner{M, N}, input::Vector{Float64}) where {M, N}
    @assert size(input) == (M,)
    return learner.weights * input + learner.biases
end

implementation (generic function with 1 method)

We will, of course, also need to incorporate some nonlinearity by introducing the sigmoid activation function, which can be described as its own separate learner. Note that, since this activation function has no trainable parameters, the learner's parameter space is an arbitrary singleton set $\{\star\}$. Its implementation function is given by $I(\star, x) = (\sigma(x_i))_i$ for all $x \in \mathbb{R}^N$, where $\sigma : \mathbb{R} \to \mathbb{R}$ denotes the sigmoid function. We represent a sigmoid learner from $\mathbb{R}^N$ to $\mathbb{R}^N$ by the `SigmoidLearner{N}` type.

In [3]:
σ(x::Float64) = 1 / (1 + exp(-x))
Dσ(x::Float64) = σ(x) * (1 - σ(x))

struct SigmoidLearner{N} <: Learner{N, N}
end

function implementation(learner::SigmoidLearner{N}, input::Vector{Float64}) where {N}
    @assert size(input) == (N,)
    return σ.(input)
end

implementation (generic function with 2 methods)

### How a Learner Learns (Update Functions)
Now, having created `LinearLearner{M, N}` and `SigmoidLearner{N}`, we need to give our learners a way to learn from observation. This is accomplished by an update function $U : P \times A \times B \to P$ that takes a current parameter $p \in P$ and an observation $(a, b) \in A \times B$ and gives an updated parameter $U(p, a, b) \in P$ that should—although this is not strictly necessary—improve on the current approximation.

We need to introduce a learning rate $\epsilon > 0$ and a function $e : \mathbb{R} \times \mathbb{R} \to \mathbb{R}$ such that $E_I(p, a, b) = \sum_j e(I_j(p, a) ,b_j)$ computes the error between a predicted output $I(p, a)$ and an actual output $b$. Then, we compute the derivative of this error with respect to each the parameters and nudge this parameters to reduce this error. Mathematically, our update function can be generally formulated as
$$ U(p, a, b) = p - \epsilon \nabla_p E_I(p, a, b)$$
for all $p \in P$, $a \in A$, and $b \in B$. Below, we specialise this formula to `LinearLearner{M, N}` using quadratic loss $e(x, y) = \frac{1}{2} (x - y)^2$. Observe that, because `SigmoidLearner{N}` has no meaningful parameterrs, its update function does not actually change anything.

In [4]:
ϵ = 0.1

function update(
        learner::LinearLearner{M, N},
        input::Vector{Float64},
        expected::Vector{Float64}
) where {M, N}
    @assert size(input) == (M,)
    @assert size(expected) == (N,)

    predicted = implementation(learner, input)
    weights = learner.weights - ϵ * (predicted - expected) * input'
    biases = learner.biases - ϵ * (predicted - expected)

    return LinearLearner{M, N}(weights, biases)
end

function update(
    learner::SigmoidLearner{N},
    input::Vector{Float64},
    expected::Vector{Float64}
) where {N}
    @assert size(input) == (N,)
    @assert size(expected) == (N,)

    return learner
end

update (generic function with 2 methods)

The introduction of these update functions allows us to already implement a function for training a learner. This is accomplished by the `train` function.

In [5]:
step(learner, datum) = update(learner, datum.input, datum.output)
train(learner, data, epochs) = foldl(step, Iterators.repeat(data, epochs); init=learner)

train (generic function with 1 method)

We can verify that the update function for our `LinearLearner{M, N}` is indeed improving the parameter choices by testing it on a simple example. Here, we generate a linear target function from $\mathbb{R}^2$ to $\mathbb{R}^2$ and a `LinearLearner{2, 2}` find the underlying weights and biases.

In [6]:
Random.seed!(0)

input_size = 2
output_size = 2
samples = 100
epochs = 100

# Target Function
W = round.(rand(output_size, input_size); digits=2)
b = round.(rand(output_size); digits=2)
f(x) = W * x + b

# Training
data = [(input=Vector(x), output=f(Vector(x))) for x in eachcol(rand(input_size, samples))]
learner = LinearLearner{2, 2}()
result = train(learner, data, epochs)

println("Target Weights: $(W)")
println("Target Biases: $(b)")
println()
println("Trained Weights: $(round.(result.weights; digits=2))")
println("Trained Biases: $(round.(result.biases; digits=2))")

Target Weights: [0.41 0.86; 0.07 0.09]
Target Biases: [0.66, 0.12]

Trained Weights: [0.41 0.86; 0.07 0.09]
Trained Biases: [0.66, 0.12]


### Composing Learners (Request Functions)
So, we are ready to combine our learners to create arbitrarily complex networks, right? Below, we use the notation $(P, I, U, r)$ and $(Q, J, V, s)$ to represent a pair of learners from $A$ to $B$ and from $B$ to $C$, respectively.
We want to find their composite
$$ (Q, J, V, s) \circ (P, I, U, r). $$
This means that we need to give a parameter space, implementation function, update function, and (yet to be introduced) request function for the resulting learner. Let's try and create a `CompositeLearner{M, N}` type that represents this composition of multiple learners.

In [7]:
struct CompositeLearner{M, H, N} <: Learner{M, N}
    right::Learner{M, H}
    left::Learner{H, N}
end

function Base.:∘(left::Learner{M, H}, right::Learner{H, N}) where {M, H, N}
    return CompositeLearner{M, H, N}(right, left)
end

The choice of parameter space is straightforward and, because we need to store the details necessary to parameterise both learners, we choose $P \circ Q = P \times Q$ to be the parameter space of their composite. Similarly, the implementation function is given simply by the function composition of $I$ and $J$, that is, we have
$$ (I \circ J)((p, q), a) = J(q, I(p, a)) $$
for all $(p, q) \in P \times Q$ and $a \in A$.

In [8]:
function implementation(
    learner::CompositeLearner{M, H, N},
    input::Vector{Float64}
) where {M, H, N}
    @assert size(input) == (M,)
    return implementation(learner.left, implementation(learner.right, input))
end

implementation (generic function with 3 methods)

A problem arises when trying to create a composite update function. Recall that the update function $U$ requires an observation $(a, b) \in A \times B$ and the update function $V$ requires an observation $(b, c) \in B \times C$. But, when training our composite learner—whose input space is $A$ and output space is $C$—we only receive observations $(a, c) \in A \times C$. A natural choice for the observation from $B \times C$ is $(I(p, a), c)$ because $I(p, a) \in B$ is the intermediate value after applying only the first learner $(P, I, U, r)$.

The request function $s : Q \times B \times C \to B$ is introduced to provide the observation $(a, s(q, b, c))$ from $A \times B$. This function should be designed in a such a way that it usually returns an element $b' = s(q, b, c) \in B$ with $J(q, b')$ closer to $c$ than the original $J(q, b)$. This is achieved by setting
$$ s(q, b, c) = f_b (\nabla_a E_J(q, b, c)) $$
where $f_b$ denotes the inverse of the componentwise function $\frac{\partial e}{\partial x}(b_i, \cdot)$. Note that the request function for $(P, I, U, r)$ is defined similarly.

In [9]:
function request(
    learner::LinearLearner{M, N},
    input::Vector{Float64},
    expected::Vector{Float64}
) where {M, N}
    @assert size(input) == (M,)
    @assert size(expected) == (N,)

    predicted = implementation(learner, input)
    return input - transpose(learner.weights) * (predicted - expected)
end

function request(
    learner::SigmoidLearner{N},
    input::Vector{Float64},
    expected::Vector{Float64}
) where {N}
    @assert size(input) == (N,)
    @assert size(expected) == (N,)

    predicted = implementation(learner, input)
    return input - (predicted - expected) .* Dσ.(input)
end

request (generic function with 2 methods)

After defining the request functions for our `LinearLearner{M, N}` and `SigmoidLearner{N}`, we are now able to implement the update function for `CompositeLearner{M, H, N}`.

In [10]:
function update(
    learner::CompositeLearner{M, H, N},
    input::Vector{Float64},
    expected::Vector{Float64}
) where {M, H, N}
    @assert size(input) == (M,)
    @assert size(expected) == (N,)
    
    forward = implementation(learner.right, input)
    left = update(learner.left, forward, expected)

    backward = request(learner.left, forward, expected)
    right = update(learner.right, input, backward)

    return left ∘ right
end

update (generic function with 3 methods)

But, if we were to create and train a complicated learner, we would encounter an error. Namely, we have not yet implemented a request function for the `CompositeLearner{M, H, N}` type. The natural choice for this request function is
$$ r(p, a, s(q, I(p, a), c)). $$

In [11]:
function request(
    learner::CompositeLearner{M, H, N},
    input::Vector{Float64},
    expected::Vector{Float64}
) where {M, H, N}
    @assert size(input) == (M,)
    @assert size(expected) == (N,)

    forward = implementation(learner.right, input)
    return request(learner.right, input, request(learner.left, forward, expected))
end

request (generic function with 3 methods)

Again, we repeat a small sanity check to ensure that these composite learners are able to successfully learn.

In [12]:
Random.seed!(0)

input_size = 2
output_size = 2
samples = 100
epochs = 100

# Target Function
W = round.(rand(output_size, input_size); digits=2)
b = round.(rand(output_size); digits=2)
f(x) = σ.(W * x + b)

# Training
data = [(input=Vector(x), output=f(Vector(x))) for x in eachcol(rand(input_size, samples))]
learner = SigmoidLearner{2}() ∘ LinearLearner{2,2}()
result = train(learner, data, epochs)

println("Target Weights: $(W)")
println("Target Biases: $(b)")
println()
println("Trained Weights: $(round.(result.right.weights; digits=2))")
println("Trained Biases: $(round.(result.right.biases; digits=2))")

Target Weights: [0.41 0.86; 0.07 0.09]
Target Biases: [0.66, 0.12]

Trained Weights: [0.42 0.85; 0.09 0.11]
Trained Biases: [0.66, 0.1]
